<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/biz/sma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime, time, requests
from google.colab import userdata

WAIT_SECOND = 3
SMA_SHORT_DAYS = 2
SMA_LONG_DAYS = 5
THRESHOLD = 0.00002
CAPITAL = 10000

### Change to your ProfitView webhook id
RESTFUL_ID = userdata.get('profitView')
BASE_URL = 'https://profitview.net/trading/signal/{}/'.format(RESTFUL_ID)

url_trade = BASE_URL + 'trade'
url_quote = BASE_URL + 'quote'

In [ ]:
def run(marketData, tradeStats):
    # Process the current quote (bid/ask)
    quote = requests.get(url_quote).json()
    bid = quote['data']['bid'][0]
    ask = quote['data']['ask'][0]

    quoteMsg = getQuoteMsg(bid, ask, marketData['bid'], marketData['ask'])
    marketData['bid'] = bid
    marketData['ask'] = ask

    # Process the current trade
    trade = requests.get(url_trade).json()
    price = trade['data']['price']
    trades = marketData['trades']

    if len(trades) < SMA_LONG_DAYS:
        trades.append(price)
        print(trades)
    else:
        trades = trades[1:]
        trades.append(price)

    marketData['trades'] = trades

    if len(trades) >= SMA_LONG_DAYS:
        long_trades = trades[len(trades)-SMA_LONG_DAYS:]
        short_trades = trades[len(trades)-SMA_SHORT_DAYS:]

        sma_long =  sum(long_trades) / SMA_LONG_DAYS
        sma_short = sum(short_trades) / SMA_SHORT_DAYS
        sma_diff = (sma_short - sma_long) / price

        # Determine signal
        if sma_diff > THRESHOLD: signal = 1
        elif sma_diff < -THRESHOLD: signal = -1
        else: signal = 0

        print("{} {}: sma_long={:,.2f}, sma_short={:,.2f}, sma_diff={:,.2f}, signal={}".format(
            quoteMsg, trades, sma_long, sma_short, sma_diff, signal))

        calcMarketValue(quote, tradeStats)

        if tradeStats['open_qty'] == 0:
            if signal == 1:
              openLongPosition(quote, tradeStats)
            elif signal == -1:
              openShortPosition(quote, tradeStats)

        elif tradeStats['open_qty'] > 0:
            if signal == 0:
              unwindLongPosition(quote, tradeStats)
            elif signal == -1:
              unwindLongPosition(quote, tradeStats)
              openShortPosition(quote, tradeStats)

        elif tradeStats['open_qty'] < 0:
            if signal == 0:
              unwindShortPosition(quote, tradeStats)
            elif signal == 1:
              unwindShortPosition(quote, tradeStats)
              openLongPosition(quote, tradeStats)


def logTradeStats(tradeStats):
    print("tradeStats: cash={:,.2f}, open_qty={:,.2f}, open_value={:,.2f}, market_value={:,.2f}, open_pnl={:,.2f}, portfolio_value={:,.2f}, portfolio_pnl={:,.2f}".format(
        tradeStats['cash'], tradeStats['open_qty'], tradeStats['open_value'], tradeStats['market_value'], tradeStats['open_pnl'], tradeStats['portfolio_value'], tradeStats['portfolio_pnl']))


def getQuoteMsg(bid, ask, prevBid, prevAsk):
    if bid == 0 or bid == prevBid: bidDir = 'S'
    elif bid > prevBid: bidDir = 'U'
    elif bid < prevBid: bidDir = 'D'

    if ask == 0 or ask == prevAsk: askDir = 'S'
    elif ask > prevAsk: askDir = 'U'
    elif ask < prevAsk: askDir = 'D'

    return "{}{} | {}{}".format(bid, bidDir, ask, askDir)


def logTrade(trade):
    print("Trade: {}: {} {}@{}".format(toDateTime(trade['data']['time']), trade['data']['side'], trade['data']['size'], trade['data']['price']))


def openLongPosition(quote, tradeStats):
    print("Open long position")
    tradeStats['open_qty'] = openTrade(quote, 'Buy', tradeStats['cash'])
    tradeStats['open_value'] = tradeStats['cash']
    tradeStats['cash'] = 0
    calcMarketValue(quote, tradeStats)


def openShortPosition(quote, tradeStats):
    print("Open short position")
    tradeStats['open_qty'] = openTrade(quote, 'Sell', tradeStats['cash'])
    tradeStats['open_value'] = -tradeStats['cash']
    tradeStats['cash'] -= tradeStats['open_value']
    calcMarketValue(quote, tradeStats)


def unwindLongPosition(quote, tradeStats):
    print("unwind long position")
    tradeStats['cash'] = unwindTrade(quote, 'Sell', tradeStats['open_qty'])
    tradeStats['open_qty'] = 0
    tradeStats['open_value'] = 0
    calcMarketValue(quote, tradeStats)


def unwindShortPosition(quote, tradeStats):
    print("unwind short position")
    tradeStats['cash'] += unwindTrade(quote, 'Buy', tradeStats['open_qty'])
    tradeStats['open_qty'] = 0
    tradeStats['open_value'] = 0
    calcMarketValue(quote, tradeStats)


def getTouchPx(quote, isBuy):
    return quote['data']['ask'][0] if isBuy else quote['data']['bid'][0]


def openTrade(quote, side, amt):
    tradePrice = getTouchPx(quote, side=='Buy')
    tradeQty = amt / tradePrice
    print("{} {:,.2f} @ {}".format(side, tradeQty, tradePrice))

    return tradeQty if side=='Buy' else -tradeQty


def unwindTrade(quote, side, tradeQty):
    tradePrice = getTouchPx(quote, side=='Buy')
    amt = tradePrice * tradeQty
    print("{} {:,.2f} @ {}".format(side, tradeQty, tradePrice))

    return amt


def calcMarketValue(quote, tradeStats):
    # market_value
    if tradeStats['open_qty'] == 0: tradeStats['market_value'] = 0

    if tradeStats['open_qty'] > 0:
      tradeStats['market_value'] = tradeStats['open_qty'] * quote['data']['bid'][0]
    else:
      tradeStats['market_value'] = tradeStats['open_qty'] * quote['data']['ask'][0]

    tradeStats['open_pnl'] = tradeStats['market_value'] - tradeStats['open_value']
    tradeStats['portfolio_value'] = tradeStats['cash'] + tradeStats['market_value']
    tradeStats['portfolio_pnl'] = tradeStats['portfolio_value'] - CAPITAL

    logTradeStats(tradeStats)


def toDateTime(unixTime):
    return datetime.datetime.fromtimestamp(unixTime / 1e3)


In [ ]:
marketData = {}
marketData['trades'] = []
marketData['bid'] = 0
marketData['ask'] = 0

In [ ]:
tradeStats = {}
tradeStats['cash'] = CAPITAL
tradeStats['open_qty'] = 0
tradeStats['open_value'] = 0
tradeStats['market_value'] = 0
tradeStats['open_pnl'] = 0
tradeStats['portfolio_value'] = CAPITAL
tradeStats['portfolio_pnl'] = 0


while True:
    run(marketData, tradeStats)
    time.sleep(WAIT_SECOND)